In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, mutual_info_classif

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression

In [48]:
import warnings
warnings.filterwarnings("ignore")

# Data

In [49]:
train = pd.read_csv("location_train.csv")
test = pd.read_csv("location_test.csv")

In [50]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Columns: 448 entries, ID to 446
dtypes: int64(448)
memory usage: 13.7 MB


In [51]:
train.head()

,ID,class,1,2,3,4,5,6,7,8,...,437,438,439,440,441,442,443,444,445,446
0,0,11,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,8,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [52]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Columns: 447 entries, ID to 446
dtypes: int64(447)
memory usage: 3.4 MB


In [53]:
test.head()

,ID,1,2,3,4,5,6,7,8,9,...,437,438,439,440,441,442,443,444,445,446
0,4000,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,4001,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,4002,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4003,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,4004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [54]:
X_train = train.drop(["ID", "class"], axis=1)
y_train = train["class"]

X_test = test.drop(["ID"], axis=1)

# Model selection

In [56]:
# Grid Search

cv = 5         # number of folds
verbose = 1     # information shown during training

## Logistic Regression

In [57]:
parameters = {
    "penalty":["l1", "l2", "elasticnet", "none"], 
    "C":[0.001, 0.01 ,0.1, 1, 5, 10, 100],
    "class_weight":["balanced"],
    "solver":["saga"]}
model = GridSearchCV(LogisticRegression(), parameters, cv=cv, verbose=verbose, scoring="f1_weighted")
model.fit(X_train_enc, y_train)

results = pd.DataFrame(model.cv_results_)
results = results[["param_penalty", "param_C", "param_class_weight", "param_solver", "mean_test_score"]]
results.sort_values(["mean_test_score"], ascending=False).head(10)

Fitting 5 folds for each of 28 candidates, totalling 140 fits


,param_penalty,param_C,param_class_weight,param_solver,mean_test_score
9,l2,0.1,balanced,saga,0.686665
5,l2,0.01,balanced,saga,0.681162
7,none,0.01,balanced,saga,0.666746
13,l2,1,balanced,saga,0.666624
3,none,0.001,balanced,saga,0.666578
11,none,0.1,balanced,saga,0.666271
21,l2,10,balanced,saga,0.666015
17,l2,5,balanced,saga,0.665645
19,none,5,balanced,saga,0.664722
23,none,10,balanced,saga,0.664455


# Final model

In [58]:
best_model = model.best_estimator_
best_model.fit(X_train_enc, y_train)

LogisticRegression(C=0.1, class_weight='balanced', solver='saga')

In [59]:
predictions = pd.DataFrame(test["ID"])
predictions["class"] = best_model.predict(X_test_enc)

In [60]:
predictions.to_csv("submission.csv", index=False)